# Architecture Autoencoder + Perceptron
Reload an autoencoder created from the file 3-autoencoder.ipynb and train it's encoded data into a network of perceptrons.

In [ ]:
import matplotlib.pyplot as matPlt
import random
import numpy as np

import tensorflow as tf
import keras
from keras import layers

from tqdm.keras import TqdmCallback
from sklearn.metrics import confusion_matrix
from shapely.geometry import Polygon, box

import rasterio
from rasterio import plot as rastPlt
from rasterio.plot import reshape_as_raster
from rasterio.mask import mask as rasterMask

import junodch_utils_read_img as utils

# Data preparation
### Fetch data from file

In [ ]:
folderName = "img/Sokoto/"
pathSatellite = folderName + "Sentinel-2.tif"
pathNight = folderName + "Night_VIIRS.tif"
pathValidation = folderName + "Population_GHSL.tif"

aoi = utils.getImgBorder(pathSatellite)

# Fetch coords
dataCoords, dataRadiance = utils.getTilesCoordsPerimeter(pathNight, area=aoi)
print('Tiles:',dataCoords.shape[0])

#### Prepare Validation

In [ ]:
with rasterio.open(pathValidation) as f:
  testTile, _ = rasterMask(f, [Polygon(dataCoords[1000])], crop=True)
print('Validation shape:',testTile.shape) # shape sample

print('Process validation...')
getValid = lambda data : [ int(250/255 < img.max()) for img in data ]
resultValid = utils.scanSatellite(pathValidation, dataCoords, getValid, batch=1000, res=testTile.shape[1])

#### Fetch Images

In [ ]:
lightMask = (dataRadiance>0) & (resultValid == 1)
lightCoords = dataCoords[lightMask]
darkMask = (dataRadiance==0) & (resultValid == 0)

idxDarkData = np.random.choice(np.arange(dataCoords.shape[0])[darkMask], len(lightCoords), replace=False)
darkCoords = dataCoords[idxDarkData]

trainCoords = np.concatenate((lightCoords, darkCoords))

with rasterio.open(pathSatellite) as f:
  trainData, _ = utils.coordsToImgsFormated(f, trainCoords, res=64)
print(trainData.shape)

print('Light Tile:',len(lightCoords))
print('dark Tile:',len(darkCoords))
print('Total train',trainData.shape)

Preprocess the data with the encoder part of the autoencoder.

# Autoencoder + Perceptron

In [ ]:
autoencoder = keras.models.load_model('model/autoencoder_64px_encoder_1024')
encoder = keras.Model(inputs=autoencoder.inputs, outputs=autoencoder.get_layer('encoder').output)

train = encoder.predict(trainData)
validation = np.concatenate((dataRadiance[lightMask], dataRadiance[idxDarkData]))

In [ ]:
# Input encoder
input_shape = keras.Input(shape=train.shape[1:])

optimizer = keras.optimizers.Adam(
  learning_rate=0.0001,
  beta_1=0.9,
  beta_2=0.999,
)
lossFunction = keras.losses.MeanSquaredError() # l2

activationFunction = 'relu'

earlyStop = tf.keras.callbacks.EarlyStopping(monitor='loss', min_delta=0, patience=5)

cnn = layers.Dense(256, activation=activationFunction)(input_shape)
cnn = layers.Dense(64, activation=activationFunction)(cnn)
cnn = layers.Dense(1, activation='sigmoid')(cnn)

model = keras.Model(input_shape, cnn)
model.compile(optimizer=optimizer, loss=lossFunction)

result = model.fit(
  x=train[::2],
  y=validation[::2],
  epochs=30,
  batch_size=20,
  shuffle=True,
  verbose=0,
  validation_data=(train[1::2],validation[1::2]),
  callbacks=[
    TqdmCallback(verbose=1), # Concise display progression
    earlyStop,
  ],
)

In [ ]:
matPlt.plot(result.history['loss'][:], label='Training')
matPlt.plot(result.history['val_loss'][:], label='test')
model.summary()

## Analyse model

In [ ]:
print('Process score...')
getScore = lambda data : model.predict(encoder.predict(data, verbose=0), verbose=0).flatten()
result = utils.scanSatellite(pathSatellite, dataCoords, getScore, batch=1000, res=64)

### Confusion Matrix. 

In [ ]:
threshold = 0.2

print('Process confustion matrix...')
print('total',len(result))
resultTest = (np.asarray(result) > threshold).astype(int)
confusionMatrix = confusion_matrix(resultValid, resultTest)
print(confusionMatrix)
tp = confusionMatrix[1][1]
fp = confusionMatrix[0][1]
fn = confusionMatrix[1][0]
print('f-score:',round(tp / (tp + (fp + fn)/2) * 100, 2),"%")
print('tp / fp:',round(tp / (tp + fp) * 100,2),"%")

In [ ]:
resultImg, resultMeta = utils.mapResultOnImg(pathNight, dataCoords, resultTest, resultValid)

fig, axs = matPlt.subplots(1,3, dpi=240)
with rasterio.open(pathSatellite) as s: utils.displayTiles([s.read()], [s.transform],axs[0])

axs[2].set_xlim(axs[0].get_xlim())
axs[2].set_ylim(axs[0].get_ylim())

with rasterio.open(pathValidation) as p: rastPlt.show(p, ax=axs[2])

axs[1].set_xlim(axs[0].get_xlim())
axs[1].set_ylim(axs[0].get_ylim())

utils.displayTiles([resultImg], [resultMeta], axs[1])

## save result

In [ ]:
model.save('model/perceptrons_256_64_V1')

In [ ]:
img = np.copy(resultImg)
img = img[0:3,:,:]
img = img.transpose([1, 2, 0])
with rasterio.open(pathNight) as f:
  profile = f.profile
img = reshape_as_raster(img)
profile.update(count=3)
print(profile)
print(img.shape)

In [ ]:
with rasterio.open(folderName + 'autoencoder_perceptron.tif', 'w', **profile) as f:
  f.write(img)
